In [3]:
import tensorflow as tf
import keras
import numpy as np
import csv
import cv2

# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
from os.path import exists

raw = []
camera_correction = 0.2 
with open('data/driving_log.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = next(csv_reader)
    for line in csv_reader:
        steering = float(line[3])
        path = 'data/' + line[0]
        if exists(path):
            raw.append(('data/' + line[0], steering))
        path = 'data/' + line[1][1:]
        if exists(path):
            raw.append(('data/' + line[1][1:], steering-camera_correction))
        path = 'data/' + line[2][1:]
        if exists(path):
            raw.append(('data/' + line[2][1:], steering+camera_correction))

from sklearn.model_selection import train_test_split
train_raw, validation_raw = train_test_split(raw, test_size=0.2)

In [5]:
from sklearn.utils import shuffle

def sample_generator(raw, batch_size=32):
    N = len(raw)
    while True:
        raw = shuffle(raw)
        for offset in range(0, N, batch_size):
            batch = raw[offset:offset+batch_size]
            imgs = []
            steerings = []
            for path, steering in batch:
                img = cv2.imread(path)
                imgs.append(img)
                steerings.append(steering)
                imgs.append(cv2.flip(img, 1))
                steerings.append(-steering)
            
            yield np.array(imgs), np.array(steerings)

train_sample_generator = sample_generator(train_raw, 32)
validation_sample_generator = sample_generator(validation_raw, 32)

In [6]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Conv2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

In [11]:
def myModel():
    model = Sequential()

    model = addPreProcess(model)

    model.add(Conv2D(24,5, padding='same', strides=(2,2), activation='relu', name='2d1'))
    model.add(Conv2D(36,5, padding='same', strides=(2,2), activation='relu', name='2d2'))
    model.add(Conv2D(48,5, padding='same', strides=(2,2), activation='relu', name='2d3'))
    model.add(Conv2D(64,3, padding='same', activation='relu', name='2d4'))
    model.add(Conv2D(64,3, padding='same', activation='relu', name='2d5'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    
    return model

def addPreProcess(model):
    model.add(Lambda(lambda x: (x-128)/255.0, input_shape=(160, 320, 3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))
    return model

model = myModel()

In [13]:
model.compile(loss='mse', optimizer='adam')

history_object = model.fit_generator(train_sample_generator, steps_per_epoch=np.ceil(len(train_raw)/16), validation_data=validation_sample_generator, \
                 epochs=5, verbose=1)

model.save('model.h5')
print(history_object.history.keys())
print('Loss')
print(history_object.history['loss'])
print('Validation Loss')
print(history_object.history['val_loss'])

<ipython-input-13-3282546cf85f>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_object = model.fit_generator(train_sample_generator, steps_per_epoch=np.ceil(len(train_raw)/16), validation_data=validation_sample_generator, \


Epoch 1/5


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential_2/2d1/Conv2D' defined at (most recent call last):
    File "/opt/miniconda3/envs/chaojie/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/miniconda3/envs/chaojie/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/miniconda3/envs/chaojie/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/opt/miniconda3/envs/chaojie/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
      app.start()
    File "/opt/miniconda3/envs/chaojie/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 612, in start
      self.io_loop.start()
    File "/opt/miniconda3/envs/chaojie/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/miniconda3/e